In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob

In [2]:
DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

In [3]:
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

In [4]:
from keras.preprocessing.text import Tokenizer

MAX_WORD_NUMS = 10000

word_tokenizer = Tokenizer(MAX_WORD_NUMS)
word_tokenizer.fit_on_texts(question_data["words"])

Using TensorFlow backend.


In [5]:
word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_WORD_NUMS]].values
    ),
    axis=0
)
word_embedding_data.shape

(10001, 300)

In [6]:
from keras.preprocessing.sequence import pad_sequences

WORD_SEQ_LEN = 30

def gen_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=WORD_SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_data(train_data)
test_word1, test_word2 = gen_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape

((254386, 30), (254386, 30), (172956, 30), (172956, 30))

In [7]:
label = train_data["label"].values

In [1]:
from keras.layers import Input, Embedding, Conv1D, GlobalAveragePooling1D, Lambda, Dropout, BatchNormalization, Dense, K
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.merge import concatenate

CONV_LEN_1 = 128
CONV_LEN_2 = 128
CONV_LEN_3 = 128
CONV_LEN_4 = 128
CONV_LEN_5 = 128
CONV_LEN_6 = 128
CONV_LEN = CONV_LEN_1 + CONV_LEN_2 + CONV_LEN_3 + CONV_LEN_4 + CONV_LEN_5 + CONV_LEN_6
DROP_RATE = 0.7
DENSE_SIZE = 256
BATCH_SIZE = 2048
NUM_EPOCHES = 100

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module(mname)
  File "C:\ProgramData\Anaconda3\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 978, in _gcd_import
  File "<frozen importlib._bootstrap>", line 961, in _find_and_load
  File "<frozen importlib._bootstrap>", line 950, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 648, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 560, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 205, in _call_with_frames_removed
ImportError: DLL load failed: 页面文件太小，无法完成操作。

During handling of the above exception, another exception occurred:

Tracebac

In [ ]:
from sklearn.model_selection import train_test_split

train_word1, dev_word1, train_word2, dev_word2, train_y, dev_y = train_test_split(
    word1, word2, train_data["label"].values,
    test_size=0.2
)

word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")

embedding_layer = Embedding(
    input_dim=word_embedding_data.shape[0],
    output_dim=word_embedding_data.shape[1],
    weights=[word_embedding_data],
    input_length=WORD_SEQ_LEN,
    trainable=False
)

word_vector1 = embedding_layer(word_input1)
word_vector2 = embedding_layer(word_input2)

conv1 = Conv1D(filters=128, kernel_size=1, padding="same", activation="relu")
conv1a = conv1(word_vector1)
conv1a = GlobalAveragePooling1D()(conv1a)
conv1b = conv1(word_vector2)
conv1b = GlobalAveragePooling1D()(conv1b)

conv2 = Conv1D(filters=128, kernel_size=2, padding="same", activation="relu")
conv2a = conv2(word_vector1)
conv2a = GlobalAveragePooling1D()(conv2a)
conv2b = conv2(word_vector2)
conv2b = GlobalAveragePooling1D()(conv2b)

conv3 = Conv1D(filters=128, kernel_size=3, padding="same", activation="relu")
conv3a = conv3(word_vector1)
conv3a = GlobalAveragePooling1D()(conv3a)
conv3b = conv3(word_vector2)
conv3b = GlobalAveragePooling1D()(conv3b)

conv4 = Conv1D(filters=128, kernel_size=4, padding="same", activation="relu")
conv4a = conv4(word_vector1)
conv4a = GlobalAveragePooling1D()(conv4a)
conv4b = conv4(word_vector2)
conv4b = GlobalAveragePooling1D()(conv4b)

conv5 = Conv1D(filters=128, kernel_size=5, padding="same", activation="relu")
conv5a = conv5(word_vector1)
conv5a = GlobalAveragePooling1D()(conv5a)
conv5b = conv5(word_vector2)
conv5b = GlobalAveragePooling1D()(conv5b)

conv6 = Conv1D(filters=128, kernel_size=6, padding="same", activation="relu")
conv6a = conv6(word_vector1)
conv6a = GlobalAveragePooling1D()(conv6a)
conv6b = conv6(word_vector2)
conv6b = GlobalAveragePooling1D()(conv6b)

merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])

diff = Lambda(lambda x: x[0] - x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])
mult = Lambda(lambda x: x[0] * x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])

merge = concatenate([diff, mult])

x = Dropout(DROP_RATE)(merge)
x = BatchNormalization()(x)
x = Dense(DENSE_SIZE, activation="relu")(x)

x = Dropout(DROP_RATE)(x)
x = BatchNormalization()(x)
pred = Dense(1, activation="sigmoid")(x)

model = Model(
    inputs = [word_input1, word_input2],
    outputs = pred
)
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["acc"]
)

early_stop = EarlyStopping("val_loss", patience=10)
check_point = ModelCheckpoint(
    "./log/cnn_%02d.{epoch:02d}_{val_loss:.3f}.hdf5" % (i + 1),
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True
)

model_res = model.fit(
    x=[train_word1, train_word2],
    y=train_y,
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHES,
    validation_data=([dev_word1, dev_word2], dev_y),
    shuffle=True,
    callbacks=[early_stop, check_point]
)

In [8]:
# from sklearn.model_selection import StratifiedKFold

# pred_collect = []

# for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10).split(X=word1, y=label)):
#     train_word1, train_word2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]
#     dev_word1, dev_word2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]
    
#     word_input1 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
#     word_input2 = Input(shape=(WORD_SEQ_LEN,), dtype="int32")
    
#     embedding_layer = Embedding(
#         input_dim=word_embedding_data.shape[0],
#         output_dim=word_embedding_data.shape[1],
#         weights=[word_embedding_data],
#         input_length=WORD_SEQ_LEN,
#         trainable=False
#     )
    
#     word_vector1 = embedding_layer(word_input1)
#     word_vector2 = embedding_layer(word_input2)
    
#     conv1 = Conv1D(filters=128, kernel_size=1, padding="same", activation="relu")
#     conv1a = conv1(word_vector1)
#     conv1a = GlobalAveragePooling1D()(conv1a)
#     conv1b = conv1(word_vector2)
#     conv1b = GlobalAveragePooling1D()(conv1b)
    
#     conv2 = Conv1D(filters=128, kernel_size=2, padding="same", activation="relu")
#     conv2a = conv2(word_vector1)
#     conv2a = GlobalAveragePooling1D()(conv2a)
#     conv2b = conv2(word_vector2)
#     conv2b = GlobalAveragePooling1D()(conv2b)
    
#     conv3 = Conv1D(filters=128, kernel_size=3, padding="same", activation="relu")
#     conv3a = conv3(word_vector1)
#     conv3a = GlobalAveragePooling1D()(conv3a)
#     conv3b = conv3(word_vector2)
#     conv3b = GlobalAveragePooling1D()(conv3b)
    
#     conv4 = Conv1D(filters=128, kernel_size=4, padding="same", activation="relu")
#     conv4a = conv4(word_vector1)
#     conv4a = GlobalAveragePooling1D()(conv4a)
#     conv4b = conv4(word_vector2)
#     conv4b = GlobalAveragePooling1D()(conv4b)
    
#     conv5 = Conv1D(filters=128, kernel_size=5, padding="same", activation="relu")
#     conv5a = conv5(word_vector1)
#     conv5a = GlobalAveragePooling1D()(conv5a)
#     conv5b = conv5(word_vector2)
#     conv5b = GlobalAveragePooling1D()(conv5b)
    
#     conv6 = Conv1D(filters=128, kernel_size=6, padding="same", activation="relu")
#     conv6a = conv6(word_vector1)
#     conv6a = GlobalAveragePooling1D()(conv6a)
#     conv6b = conv6(word_vector2)
#     conv6b = GlobalAveragePooling1D()(conv6b)
    
#     merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
#     merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    
#     diff = Lambda(lambda x: x[0] - x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])
#     mult = Lambda(lambda x: x[0] * x[1], output_shape=(CONV_LEN,))([merge_a, merge_b])
    
#     merge = concatenate([diff, mult])
    
#     x = Dropout(DROP_RATE)(merge)
#     x = BatchNormalization()(x)
#     x = Dense(DENSE_SIZE, activation="relu")(x)
    
#     x = Dropout(DROP_RATE)(x)
#     x = BatchNormalization()(x)
#     pred = Dense(1, activation="sigmoid")(x)
    
#     model = Model(
#         inputs = [word_input1, word_input2],
#         outputs = pred
#     )
#     model.compile(
#         optimizer="adam",
#         loss="binary_crossentropy",
#         metrics=["acc"]
#     )
    
#     early_stop = EarlyStopping("val_loss", patience=10)
#     check_point = ModelCheckpoint(
#         "./log/cnn_%02d.{epoch:02d}_{val_loss:.3f}.hdf5" % (i + 1),
#         monitor="val_loss",
#         save_best_only=True,
#         save_weights_only=True
#     )
    
#     model_res = model.fit(
#         x=[train_word1, train_word2],
#         y=train_y,
#         batch_size=BATCH_SIZE,
#         epochs=NUM_EPOCHES,
#         validation_data=([dev_word1, dev_word2], dev_y),
#         shuffle=True,
#         callbacks=[early_stop, check_point]
#     )
    
#     print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
#     model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))

#     test_pred = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)
#     pred_collect.append(pd.DataFrame(test_pred, columns=["y_pre"]))

# pd.DataFrame(pd.concat(pred_collect, axis=1).mean(axis=1), columns=["y_pre"]).to_csv("./result/pred.csv", index=False)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/100
228946/228946 [==============================] - 31s 136us/step - loss: 0.5873 - acc: 0.7152 - val_loss: 0.4358 - val_acc: 0.8015
Epoch 2/100
228946/228946 [==============================] - 29s 125us/step - loss: 0.4441 - acc: 0.7957 - val_loss: 0.4013 - val_acc: 0.8139
Epoch 3/100
228946/228946 [==============================] - 29s 128us/step - loss: 0.3982 - acc: 0.8194 - val_loss: 0.3590 - val_acc: 0.8368
Epoch 4/100
228946/228946 [==============================] - 30s 129us/step - loss: 0.3706 - acc: 0.8350 - val_loss: 0.3306 - val_acc: 0.8527
Epoch 5/100
228946/228946 [==============================] - 30s 130us/step - loss: 0.3511 - acc: 0.8447 - val_loss: 0.3133 - val_acc: 0.8615
Epoch 6/100
228946/228946 [==============================] - 30s 130us/step - loss: 0.3375 - acc: 0.8529 - val_loss: 0.3028 - val_acc: 0.8670
Epoch 7/100
22

Epoch 56/100
228946/228946 [==============================] - 30s 130us/step - loss: 0.1933 - acc: 0.9209 - val_loss: 0.2241 - val_acc: 0.9016
Epoch 57/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1909 - acc: 0.9219 - val_loss: 0.2186 - val_acc: 0.9072
Epoch 58/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1904 - acc: 0.9224 - val_loss: 0.2243 - val_acc: 0.9025
Epoch 59/100
228946/228946 [==============================] - 30s 130us/step - loss: 0.1894 - acc: 0.9222 - val_loss: 0.2264 - val_acc: 0.8998
Epoch 60/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1893 - acc: 0.9228 - val_loss: 0.2300 - val_acc: 0.8986
Epoch 61/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1876 - acc: 0.9238 - val_loss: 0.2339 - val_acc: 0.8972
Epoch 62/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1865 - acc: 0.9245 - val_loss: 0.2245 - val_acc: 0.9020

Epoch 46/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.2038 - acc: 0.9161 - val_loss: 0.2263 - val_acc: 0.9006
Epoch 47/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.2032 - acc: 0.9170 - val_loss: 0.2289 - val_acc: 0.8986
Epoch 48/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.2008 - acc: 0.9176 - val_loss: 0.2242 - val_acc: 0.9024
Epoch 49/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1998 - acc: 0.9183 - val_loss: 0.2263 - val_acc: 0.9012
Epoch 50/100
228946/228946 [==============================] - 30s 131us/step - loss: 0.1976 - acc: 0.9191 - val_loss: 0.2233 - val_acc: 0.9020
Epoch 51/100
228946/228946 [==============================] - 29s 129us/step - loss: 0.1982 - acc: 0.9189 - val_loss: 0.2260 - val_acc: 0.9011
Epoch 52/100
228946/228946 [==============================] - 30s 130us/step - loss: 0.1961 - acc: 0.9199 - val_loss: 0.2228 - val_acc: 0.9033

Epoch 26/100
228947/228947 [==============================] - 29s 129us/step - loss: 0.2373 - acc: 0.9002 - val_loss: 0.2394 - val_acc: 0.8937
Epoch 27/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2364 - acc: 0.9011 - val_loss: 0.2363 - val_acc: 0.8971
Epoch 28/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2353 - acc: 0.9018 - val_loss: 0.2384 - val_acc: 0.8944
Epoch 29/100
228947/228947 [==============================] - 30s 132us/step - loss: 0.2326 - acc: 0.9027 - val_loss: 0.2416 - val_acc: 0.8930
Epoch 30/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2311 - acc: 0.9035 - val_loss: 0.2377 - val_acc: 0.8952
Epoch 31/100
228947/228947 [==============================] - 30s 132us/step - loss: 0.2274 - acc: 0.9060 - val_loss: 0.2370 - val_acc: 0.8959
Epoch 32/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2264 - acc: 0.9058 - val_loss: 0.2331 - val_acc: 0.8991

Epoch 19/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2587 - acc: 0.8913 - val_loss: 0.2450 - val_acc: 0.8920
Epoch 20/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2566 - acc: 0.8917 - val_loss: 0.2461 - val_acc: 0.8906
Epoch 21/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2525 - acc: 0.8939 - val_loss: 0.2386 - val_acc: 0.8972
Epoch 22/100
228947/228947 [==============================] - 30s 130us/step - loss: 0.2487 - acc: 0.8962 - val_loss: 0.2416 - val_acc: 0.8937
Epoch 23/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2461 - acc: 0.8966 - val_loss: 0.2403 - val_acc: 0.8946
Epoch 24/100
228947/228947 [==============================] - 30s 131us/step - loss: 0.2434 - acc: 0.8980 - val_loss: 0.2383 - val_acc: 0.8956
Epoch 25/100
228947/228947 [==============================] - 30s 132us/step - loss: 0.2421 - acc: 0.8985 - val_loss: 0.2381 - val_acc: 0.8951

...

Epoch 51/100
228948/228948 [==============================] - 30s 129us/step - loss: 0.1989 - acc: 0.9184 - val_loss: 0.2275 - val_acc: 0.9006
Epoch 52/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.1970 - acc: 0.9195 - val_loss: 0.2291 - val_acc: 0.8998
Epoch 53/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.1941 - acc: 0.9204 - val_loss: 0.2284 - val_acc: 0.8996
Epoch 54/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.1949 - acc: 0.9204 - val_loss: 0.2293 - val_acc: 0.8990
Epoch 55/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.1937 - acc: 0.9212 - val_loss: 0.2309 - val_acc: 0.8980
Epoch 56/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.1923 - acc: 0.9216 - val_loss: 0.2239 - val_acc: 0.9026
Epoch 57/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.1910 - acc: 0.9222 - val_loss: 0.2277 - val_acc: 0.9003

Epoch 34/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2229 - acc: 0.9070 - val_loss: 0.2376 - val_acc: 0.8962
Epoch 35/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2203 - acc: 0.9092 - val_loss: 0.2348 - val_acc: 0.8978
Epoch 36/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2204 - acc: 0.9087 - val_loss: 0.2363 - val_acc: 0.8954
Epoch 37/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2177 - acc: 0.9098 - val_loss: 0.2320 - val_acc: 0.8980
Epoch 38/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2162 - acc: 0.9108 - val_loss: 0.2285 - val_acc: 0.9009
Epoch 39/100
228948/228948 [==============================] - 30s 130us/step - loss: 0.2152 - acc: 0.9109 - val_loss: 0.2300 - val_acc: 0.9009
Epoch 40/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2139 - acc: 0.9115 - val_loss: 0.2288 - val_acc: 0.8998

Epoch 21/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2519 - acc: 0.8941 - val_loss: 0.2478 - val_acc: 0.8920
Epoch 22/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2491 - acc: 0.8956 - val_loss: 0.2473 - val_acc: 0.8920
Epoch 23/100
228948/228948 [==============================] - 30s 130us/step - loss: 0.2458 - acc: 0.8968 - val_loss: 0.2466 - val_acc: 0.8918
Epoch 24/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2429 - acc: 0.8983 - val_loss: 0.2413 - val_acc: 0.8948
Epoch 25/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.2403 - acc: 0.8992 - val_loss: 0.2422 - val_acc: 0.8947
Epoch 26/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2381 - acc: 0.9004 - val_loss: 0.2414 - val_acc: 0.8954
Epoch 27/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2349 - acc: 0.9021 - val_loss: 0.2393 - val_acc: 0.8965

Epoch 6/100
228948/228948 [==============================] - 30s 131us/step - loss: 0.3333 - acc: 0.8542 - val_loss: 0.3018 - val_acc: 0.8661
Epoch 7/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.3234 - acc: 0.8601 - val_loss: 0.2914 - val_acc: 0.8715
Epoch 8/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.3128 - acc: 0.8648 - val_loss: 0.2812 - val_acc: 0.8769
Epoch 9/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.3051 - acc: 0.8682 - val_loss: 0.2852 - val_acc: 0.8702
Epoch 10/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2988 - acc: 0.8718 - val_loss: 0.2763 - val_acc: 0.8774
Epoch 11/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2916 - acc: 0.8756 - val_loss: 0.2701 - val_acc: 0.8809
Epoch 12/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2854 - acc: 0.8784 - val_loss: 0.2693 - val_acc: 0.8784
Epo

Epoch 44/100
228948/228948 [==============================] - 30s 129us/step - loss: 0.2081 - acc: 0.9143 - val_loss: 0.2215 - val_acc: 0.9029
Epoch 45/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2051 - acc: 0.9158 - val_loss: 0.2238 - val_acc: 0.8992
Epoch 46/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2041 - acc: 0.9160 - val_loss: 0.2217 - val_acc: 0.9001
Epoch 47/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2049 - acc: 0.9157 - val_loss: 0.2167 - val_acc: 0.9046
Epoch 48/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2028 - acc: 0.9167 - val_loss: 0.2184 - val_acc: 0.9033
Epoch 49/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.2000 - acc: 0.9177 - val_loss: 0.2183 - val_acc: 0.9032
Epoch 50/100
228948/228948 [==============================] - 30s 132us/step - loss: 0.1997 - acc: 0.9186 - val_loss: 0.2226 - val_acc: 0.9012